## READ ME

In [ ]:
"""
This code is not finished yet. DO NOT USE IT.

"""

' This code is not finished yet. DO NOT USE IT.\n'

## Importing Libraries

In [2]:
import numpy as np
import pandas as pd
from faker import Faker

## Directory

In [5]:
homepath = "C:/A.PROJECTS/stockprediction/"

## Bond Info

In [23]:


# Initialize Faker to generate fake company names
fake = Faker()

# Function to generate synthetic bond data with fixed coupon rates
def generate_bond_data_fixed_coupon(num_bonds, bond_type):
    np.random.seed(42)  # Ensuring reproducibility
    
    bond_data = []
    fixed_coupon_rates = [0.16, 0.18, 0.23]  # Fixed coupon rates
    
    for _ in range(num_bonds):
        if bond_type == "Treasury":
            company_name = "U.S. Treasury"  # Fixed name for treasury bonds
            price = np.random.uniform(95, 105)  # Treasury bond prices close to par
            coupon_rate = np.random.choice(fixed_coupon_rates)  # Fixed coupon rates
            time_to_maturity_years = np.random.uniform(0.5, 5)  # Short-term maturities (0.5-5 years)
            compounding = "semi-annual"
        else:  # Corporate bonds
            company_name = fake.company()  # Generate fake company name
            price = np.random.uniform(80, 110)  # More variability in corporate bond prices
            coupon_rate = np.random.choice(fixed_coupon_rates)  # Fixed coupon rates
            time_to_maturity_years = np.random.uniform(5, 30)  # Long-term corporate bonds (5-30 years)
            compounding = np.random.choice(["semi-annual", "annual", "continuous"])
        
        par_value = 100
        payment_frequency = np.random.choice([1, 2, 4])  # Annual, Semi-Annual, or Quarterly
        num_periods = int(time_to_maturity_years * payment_frequency)
        discount_rate = np.random.uniform(0.01, 0.06)  # Required yield
        day_count_convention = np.random.choice(["30/360", "actual/actual"])
        time_to_maturity_days = int(time_to_maturity_years * 365)  # Convert years to days
        
        bond_data.append([
            bond_type, company_name, price, coupon_rate, payment_frequency, par_value, 
            time_to_maturity_years, time_to_maturity_days, num_periods, discount_rate, 
            day_count_convention, compounding
        ])
    
    return bond_data

# Generate synthetic data for 10 short-term Treasury bonds and 30 long-term Corporate bonds
treasury_bonds = generate_bond_data_fixed_coupon(10, "Treasury")
corporate_bonds = generate_bond_data_fixed_coupon(30, "Corporate")

# Create a DataFrame with company names and fixed coupon rates
columns = [
    "Bond Type", "Company Name", "Bond Price", "Coupon Rate", "Coupon Payment Frequency", 
    "Face Value", "Time to Maturity (Years)", "Time to Maturity (Days)", "Number of Periods", 
    "Discount Rate (Yield)", "Day Count Convention", "Compounding Convention"
]

BondInfo_df = pd.DataFrame(treasury_bonds + corporate_bonds, columns=columns)


## Bond Price

In [24]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Parameters
num_bonds = 30  # 15 Treasury, 15 Corporate
num_days = 365 * 5  # 5 years of daily prices
start_date = datetime.today() - timedelta(days=num_days)

# Generate bond tickers
treasury_tickers = [f"T-Bond-{i+1}" for i in range(15)]
corporate_tickers = [f"Corp-Bond-{i+1}" for i in range(15)]
all_tickers = treasury_tickers + corporate_tickers

# Historical volatility estimates for real bonds (approximate)
treasury_vol = 0.002  # Low volatility
corporate_vol = 0.005  # Higher than treasuries

# Mean daily drift assumption (approximate)
treasury_drift = 0.0001
corporate_drift = 0.0003

# Generate synthetic time series data
dates = [start_date + timedelta(days=i) for i in range(num_days)]
bond_data = {"Date": dates}

for ticker in all_tickers:
    is_treasury = "T-Bond" in ticker
    drift = treasury_drift if is_treasury else corporate_drift
    vol = treasury_vol if is_treasury else corporate_vol
    
    # Simulating log-normal price evolution
    prices = [100]  # Start at par
    for _ in range(1, num_days):
        price_change = np.random.normal(drift, vol) * prices[-1]
        new_price = prices[-1] + price_change
        prices.append(new_price)
    
    bond_data[ticker] = prices

# Convert to DataFrame
BondPrice_df = pd.DataFrame(bond_data)


## Stock Info

In [18]:
import yfinance as yf
import pandas as pd
import time

# Define the fields to retrieve
general_info_fields = [
    "symbol", "shortName", "sector", "industry", "country", "fullTimeEmployees", "website", "longBusinessSummary"
]

# Function to fetch general information of tickers
def get_general_info(tickers):
    data = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.info
        data.append({field: info.get(field, "N/A") for field in general_info_fields})
    return pd.DataFrame(data)

# List of mock tickers
mock_tickers =  pd.read_excel(homepath + "data/SP500Tickers.xlsx")["Symbol"].to_list()
# Batch size
batch_size = 30

dff_list = []

# Loop through batches of tickers
for i in range(0, len(mock_tickers), batch_size):
    batch = mock_tickers[i:i + batch_size]  # Get the next 30 tickers
    print(f"Processing batch {i // batch_size + 1}: {batch}")
    
    try:
        dff = get_general_info(batch)  # Fetch general stock info
        dff_list.append(dff)  # Append DataFrame to list
        
    except Exception as e:
        print(f"Error processing batch {i // batch_size + 1}: {e}")
    
    time.sleep(5)  # Wait for 5 seconds before next batch

# Concatenate all DataFrames into a final one
StockInfo_df = pd.concat(dff_list, ignore_index=True)


Processing batch 1: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK']
Processing batch 2: ['AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX']
Processing batch 3: ['BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT']
Processing batch 4: ['CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP']
Processing batch 5: ['ED', 'STZ', 'CEG', 'COO', 'CPR